**<font color = black size=6>实验九:贝叶斯分类</font>**

In [19]:
import math
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from collections import defaultdict

**<font color = blue size=4>第一部分:实验任务</font>**

1.朴素贝叶斯

<img src='./Naive Bayes Classifier Pseudocode.jpg'>

<span style="color:purple">该数据集(train_mushroom.csv)为分类数据集，为蘑菇的特征信息以及是否有毒。包括了13个特征以及一个标签(即为label类型,代表是否有毒)。label='p'代表有毒，label='e'代表无毒。</span>

<span style="color:purple">1) 将训练数据集'train_mushroom.csv'和'test_mushroom.csv'载入并转换为你需要的格式</span>

In [20]:
# 读取数据集
train_data = pd.read_csv("train_mushroom.csv").values
test_data  = pd.read_csv("test_mushroom.csv").values

# 打印检查
print(train_data)
print(test_data)

[['s' 'n' 't' ... 'u' 'x' 'p']
 ['s' 'y' 't' ... 'g' 'x' 'e']
 ['s' 'w' 't' ... 'm' 'b' 'e']
 ...
 ['f' 'y' 'y' ... 'v' 'd' 'p']
 ['f' 'y' 'y' ... 'v' 'p' 'p']
 ['f' 'y' 'y' ... 'y' 'p' 'p']]
[['k' 'y' 'n' ... 'v' 'd' 'p']
 ['b' 'f' 'g' ... 's' 'g' 'e']
 ['b' 's' 'g' ... 'n' 'g' 'e']
 ...
 ['k' 's' 'n' ... 'v' 'l' 'p']
 ['f' 'y' 'n' ... 'v' 'd' 'p']
 ['x' 's' 'w' ... 's' 'g' 'e']]


<span style="color:purple">2) 计算每个标签值y对应的先验概率P(y)
$$P(y)=\frac{|D_y|}{|D|}$$
其中$D_y$为标签值为y的样本集合，
$|D_y|$为这个集合的样本个数；D为所有样本集合，|D|为所有样本个数

</span>

In [21]:

# 计算集合的样本个数
total_samples = len(train_data)

# 计算先验概率
prior_e = sum(train_data[:, -1] == 'e') / total_samples
prior_p = sum(train_data[:, -1] == 'p') / total_samples

# 打印检查
print(prior_e)
print(prior_p)


0.59
0.41


<span style="color:purple">3) 对于数据集中的每个特征的非重复特征值
$x_i$
，计算给定标签值y时特征值$x_i$的条件概率$P(x_i│y)$,
    $$P(x_i│y)=\frac{|D_{x_i,y}|}{|D_y|}$$
$D_{x_i,y}$为标签值为y，特征值为$x_i$的样本集合；$|D_{x_i,y}|$为该集合的样本个数
</span>

In [22]:
# 存储特征的条件概率
cond_prob = {}

# 首先遍历数据集D中的每个特征
for feature in range(train_data.shape[1] - 1):
    # 将每个特征的非重复值取出
    unique_values = np.unique(train_data[:, feature])
    
    # 存储当前特征值的条件概率
    cond_prob[feature] = {}

    # 遍历每个特征值
    for value in unique_values:
        # 计算条件概率 P(feature=value | label='e')
        prob_e = len(train_data[(train_data[:, feature] == value) & (train_data[:, -1] == 'e')]) / sum(train_data[:, -1] == 'e')

        # 计算条件概率 P(feature=value | label='q')
        prob_p = len(train_data[(train_data[:, feature] == value) & (train_data[:, -1] == 'p')]) / sum(train_data[:, -1] == 'p')

        # 存储条件概率
        cond_prob[feature][value] = {'e': prob_e, 'p': prob_p}

print(cond_prob)


{0: {'b': {'e': 0.0, 'p': 0.03902439024390244}, 'c': {'e': 0.0, 'p': 0.004878048780487805}, 'f': {'e': 0.23728813559322035, 'p': 0.36097560975609755}, 'k': {'e': 0.06779661016949153, 'p': 0.0}, 's': {'e': 0.29152542372881357, 'p': 0.05853658536585366}, 'x': {'e': 0.07796610169491526, 'p': 0.424390243902439}, 'y': {'e': 0.3254237288135593, 'p': 0.11219512195121951}}, 1: {'f': {'e': 0.013559322033898305, 'p': 0.05853658536585366}, 'g': {'e': 0.06440677966101695, 'p': 0.0}, 'n': {'e': 0.12203389830508475, 'p': 0.06341463414634146}, 's': {'e': 0.0576271186440678, 'p': 0.2682926829268293}, 'w': {'e': 0.2847457627118644, 'p': 0.1073170731707317}, 'y': {'e': 0.4576271186440678, 'p': 0.5024390243902439}}, 2: {'b': {'e': 0.02711864406779661, 'p': 0.08780487804878048}, 'c': {'e': 0.020338983050847456, 'p': 0.0}, 'e': {'e': 0.03728813559322034, 'p': 0.04390243902439024}, 'f': {'e': 0.14576271186440679, 'p': 0.00975609756097561}, 'g': {'e': 0.0, 'p': 0.12682926829268293}, 'n': {'e': 0.064406779661

<span style="color:purple">4) 编写函数，给定样本
$x=(x_1,...,x_i,...,x_d)$
以及标签y, 计算其后验概率    
    输入：样本x，标签y  
    输出：样本x对应标签y的后验概率  
    计算后验概率公式:
    $P(y)\prod_{i=1}^{d}P(x_i|y)$
    
<span style="color:purple">例:  
    特征和标签：(cap-shape, cap-surface,..., habitat), label  
    输入: [k, y, n, f, s, c, n, b, o, e, w, v, d], p  
    输出: P(label='p') 
    $\times$
     P(cap-shape='k'|label='p') 
     $\times$ ... 
     $\times$
    P(habitat='d'|label='p')</span>

In [23]:
def pro(a, index, prior):
    # 初始化为先验概率
    posterior_prob = prior

    # 遍历样本的每个特征
    for i in range(len(a)):
        
        # 获取当前特征值
        feature_value = a[i]
        
        # 计算条件概率 P(x_i|y)
        if feature_value in cond_prob[i]:
            feature_prob = cond_prob[i][feature_value][index]
        else :
            posterior_prob = 0
            break

        # feature_prob = cond_prob[i][feature_value][index]     
        # 乘以每个特征的条件概率，累积得到后验概率
        posterior_prob *= feature_prob

    return posterior_prob

<span style="color:purple">5) 对测试集中的每个样本a，利用上个步骤所编写的函数，分别求所有可能的标签对应的后验概率，后验概率更大的对应标签即为预测标签。最后与测试集本身标签进行比较计算出准确率</span>

In [24]:

def predict_label(a):
    # 计算后验概率
    posterior_prob_e = pro(a, 'e', prior_e)
    posterior_prob_p = pro(a, 'p', prior_p)

    # 判断后验概率更大的类别
    if posterior_prob_e >= posterior_prob_p:
        return 'e'
    else:
        return 'p'

# 对测试集进行预测
predictions = [predict_label(sample[:-1]) for sample in test_data]

# 计算准确率
correct_predictions = sum(predictions == test_data[:, -1])
accuracy = correct_predictions / len(test_data)

# 打印结果
print("Accuracy:", accuracy)


Accuracy: 0.34


2.引入拉普拉斯平滑

<span style="color:purple">1) 首先，请判断是否有某个特征值和某个类没有在训练集中同时出现而使得条件概率为零。如果无，则无需进行下列实验；如果有，请在上个实验的基础上引入拉普拉斯平滑</span>

<span style="color:purple">2) 引入拉普拉斯平滑后计算每个标签y对应的先验概率P(y)
$$P(y)=\frac{|D_y|+1}{|D|+N}$$
其中
$D_y$
为标签为y的样本集合；
$|D_y|$
为标签为y的集合的样本个数；D为所有样本集合；|D|为所有样本个数;N为标签取值的个数

</span>

In [25]:
# 计算先验概率

# 计算集合的样本个数
total_samples = len(train_data)

# 计算标签 'e' 的先验概率
prior_e = (len(train_data[train_data[:, -1] == 'e']) + 1) / (total_samples + len(np.unique(train_data[:, -1])))

# 计算标签 'p' 的先验概率
prior_p = (len(train_data[train_data[:, -1] == 'p']) + 1) / (total_samples + len(np.unique(train_data[:, -1])))

# 打印检查
print(prior_e)
print(prior_p)


0.5896414342629482
0.4103585657370518


<span style="color:purple">3) 计算数据集中的每个特征的非重复特征值
$x_i$
对应标签y的条件概率$P(x_i│y)$,
    $$P(x_i│y)=\frac{|D_{x_i,y}|+1}{|D_y|+N_i}$$
$D_{x_i,y}$为标签为$y$，特征为$x_i$的样本集合；$|D_{x_i,y}|$为该样本个数;$N_i$为第$i$个特征取值的个数
</span>

In [26]:
# 存储特征的条件概率
cond_prob = {}

# 首先遍历数据集D中的每个特征
for feature in range(train_data.shape[1] - 1):
    # 将每个特征的非重复值取出
    unique_values = np.unique(train_data[:,feature])
    
    # 存储当前特征值的条件概率
    cond_prob[feature] = {}

    # 遍历每个特征值
    for value in unique_values:
        # 计算条件概率 P(feature=value | label='e')
        prob_e = (len(train_data[(train_data[:, feature] == value) & (train_data[:, -1] == 'e')]) + 1) / (sum(train_data[:, -1] == 'e') + len(unique_values))

        # 计算条件概率 P(feature=value | label='q')
        prob_p = (len(train_data[(train_data[:, feature] == value) & (train_data[:, -1] == 'p')]) + 1) / (sum(train_data[:, -1] == 'p') + len(unique_values))

        # 存储条件概率
        cond_prob[feature][value] = {'e': prob_e, 'p': prob_p}
    
print(cond_prob)



{0: {'b': {'e': 0.0033112582781456954, 'p': 0.04245283018867924}, 'c': {'e': 0.0033112582781456954, 'p': 0.009433962264150943}, 'f': {'e': 0.23509933774834438, 'p': 0.35377358490566035}, 'k': {'e': 0.0695364238410596, 'p': 0.0047169811320754715}, 's': {'e': 0.28807947019867547, 'p': 0.06132075471698113}, 'x': {'e': 0.07947019867549669, 'p': 0.41509433962264153}, 'y': {'e': 0.3211920529801324, 'p': 0.11320754716981132}}, 1: {'f': {'e': 0.016611295681063124, 'p': 0.061611374407582936}, 'g': {'e': 0.0664451827242525, 'p': 0.004739336492890996}, 'n': {'e': 0.12292358803986711, 'p': 0.06635071090047394}, 's': {'e': 0.059800664451827246, 'p': 0.26540284360189575}, 'w': {'e': 0.2823920265780731, 'p': 0.10900473933649289}, 'y': {'e': 0.45182724252491696, 'p': 0.4928909952606635}}, 2: {'b': {'e': 0.029315960912052116, 'p': 0.08755760368663594}, 'c': {'e': 0.02280130293159609, 'p': 0.004608294930875576}, 'e': {'e': 0.03908794788273615, 'p': 0.04608294930875576}, 'f': {'e': 0.14332247557003258, '

<span style="color:purple">4) 对测试集中的每个样本a，利用上个步骤所编写的函数，分别求所有可能的标签对应的后验概率，后验概率更大的对应标签即为预测标签，最后与测试集本身标签进行比较计算出准确率</span>

In [27]:
def predict_label(a):
    # 计算后验概率
    posterior_prob_e = pro(a, 'e', prior_e)
    posterior_prob_p = pro(a, 'p', prior_p)

    # 判断后验概率更大的类别
    if posterior_prob_e >= posterior_prob_p:
        return 'e'
    else:
        return 'p'

# 对测试集进行预测
predictions = [predict_label(sample[:-1]) for sample in test_data]

# 计算准确率
correct_predictions = sum(predictions == test_data[:, -1])
accuracy = correct_predictions / len(test_data)

# 打印结果
print("Accuracy:", accuracy)

Accuracy: 0.74


**<font color = blue size=4>第二部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)【不要】提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可，如果交错请到讲台前联系助教，删掉之前的错误版本后再进行提交

二、实验报告截止日期： 【11月17日 14:20】
要求：  
1)文件格式为：学号-姓名.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  
3)文件命名时不需要额外添加“实验几”等额外信息，按照格式提交  
4)每周的实验报告提交地址会变化，且有时间限制，提交时间为下周的实验课开始时，请注意及时提交。

实验九(贝叶斯分类)的实验报告上交地址:https://send2me.cn/ufVNphux/T9yuatQDc00TVw  

三、课堂课件获取地址:https://www.jianguoyun.com/p/DRLiP2oQp5WhChjB86YFIAA  
实验内容获取地址:https://www.jianguoyun.com/p/DbLessAQp5WhChjD86YFIAA